In [1]:
import numpy as np

In [2]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)



In [3]:

# Linear
class ActivationLinear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

# Sigmoid
class ActivationSigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))

# TanH
class ActivationTanH:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)

# ReLU
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [4]:
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

# MSE
class Loss_MSE:
    def forward(self, y_pred, y_true):
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        self.dinputs = -2 * (y_true - y_pred) / outputs
        self.dinputs = self.dinputs / samples

# Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        return -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

    def backward(self, y_pred, y_true):
        samples = y_true.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        self.dinputs = - (y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
        self.dinputs = self.dinputs / samples

# Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        samples = y_pred.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

In [5]:
# SGD Optimizer (supports vanilla SGD, decay, and momentum)
class Optimizer_SGD:
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    def pre_update_params(self):
        """Apply learning rate decay BEFORE parameter updates"""
        if self.decay:
            self.current_learning_rate = self.learning_rate * \
                (1.0 / (1.0 + self.decay * self.iterations))

    def update_params(self, layer):
        """Update parameters - vanilla SGD or with momentum"""
        if self.momentum:
            # Initialize momentum arrays if needed
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)

            # Momentum update
            weight_updates = self.momentum * layer.weight_momentums - \
                           self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            
            bias_updates = self.momentum * layer.bias_momentums - \
                         self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
        else:
            # Vanilla SGD
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        # Apply updates
        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1


# AdaGrad Optimizer
class Optimizer_AdaGrad:
    def __init__(self, learning_rate=1.0, decay=0.0, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon

    def pre_update_params(self):
        """Apply learning rate decay"""
        if self.decay:
            self.current_learning_rate = self.learning_rate * \
                (1.0 / (1.0 + self.decay * self.iterations))

    def update_params(self, layer):
        """AdaGrad parameter update"""
        # Initialize cache if needed
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Accumulate squared gradients
        layer.weight_cache += layer.dweights ** 2
        layer.bias_cache += layer.dbiases ** 2

        # AdaGrad update
        layer.weights += -self.current_learning_rate * layer.dweights / \
                        (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / \
                       (np.sqrt(layer.bias_cache) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1

In [6]:
import nnfs
from nnfs.datasets import spiral_data

# Create the dataset
X, y = spiral_data(samples=100, classes=3)

print(X[:5])
print(X.shape)
print(y[:5])
print(y.shape)

[[-0.          0.        ]
 [ 0.0031212   0.00960669]
 [ 0.00077262  0.02018724]
 [ 0.01450635  0.02660525]
 [-0.01094888  0.03889227]]
(300, 2)
[0 0 0 0 0]
(300,)


In [7]:
#TEST 1 - Vanilla SGD with Learning Rate Decay
print("="*70)
print("Vanilla SGD with Learning Rate Decay")
print("="*70)

# Initialize network
dense1 = Layer_Dense(2, 64)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 3)
activation2 = Activation_Softmax()
loss_function = Loss_CategoricalCrossEntropy()
optimizer = Optimizer_SGD(learning_rate=1.0, decay=1e-3, momentum=0.0)

# Training loop
for epoch in range(1000):
    optimizer.pre_update_params()
    
    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    
    # Calculate loss and accuracy
    loss = loss_function.calculate(activation2.output, y)
    predictions = np.argmax(activation2.output, axis=1)
    accuracy = np.mean(predictions == y)
    
    if epoch % 100 == 0 or epoch == 999:
        print(f"Epoch {epoch:4d} | Loss: {loss:.4f} | Acc: {accuracy:.4f} | LR: {optimizer.current_learning_rate:.5f}")
    
    # Backward pass
    loss_function.backward(activation2.output, y)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

Vanilla SGD with Learning Rate Decay
Epoch    0 | Loss: 1.0986 | Acc: 0.3100 | LR: 1.00000
Epoch  100 | Loss: 1.0731 | Acc: 0.4400 | LR: 0.90909
Epoch  200 | Loss: 1.0611 | Acc: 0.4433 | LR: 0.83333
Epoch  300 | Loss: 1.0599 | Acc: 0.4367 | LR: 0.76923
Epoch  400 | Loss: 1.0586 | Acc: 0.4367 | LR: 0.71429
Epoch  500 | Loss: 1.0571 | Acc: 0.4400 | LR: 0.66667
Epoch  600 | Loss: 1.0554 | Acc: 0.4433 | LR: 0.62500
Epoch  700 | Loss: 1.0530 | Acc: 0.4400 | LR: 0.58824
Epoch  800 | Loss: 1.0490 | Acc: 0.4467 | LR: 0.55556
Epoch  900 | Loss: 1.0425 | Acc: 0.4467 | LR: 0.52632
Epoch  999 | Loss: 1.0332 | Acc: 0.4533 | LR: 0.50025


In [8]:
#TEST 2 - SGD with Momentum
print("\n" + "="*70)
print("SGD with Momentum")
print("="*70)

# Reinitialize network
dense1 = Layer_Dense(2, 64)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 3)
activation2 = Activation_Softmax()
loss_function = Loss_CategoricalCrossEntropy()
optimizer = Optimizer_SGD(learning_rate=1.0, decay=1e-3, momentum=0.9)

# Training loop
for epoch in range(1000):
    optimizer.pre_update_params()
    
    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    
    # Calculate loss and accuracy
    loss = loss_function.calculate(activation2.output, y)
    predictions = np.argmax(activation2.output, axis=1)
    accuracy = np.mean(predictions == y)
    
    if epoch % 100 == 0 or epoch == 999:
        print(f"Epoch {epoch:4d} | Loss: {loss:.4f} | Acc: {accuracy:.4f} | LR: {optimizer.current_learning_rate:.5f}")
    
    # Backward pass
    loss_function.backward(activation2.output, y)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()


SGD with Momentum
Epoch    0 | Loss: 1.0986 | Acc: 0.3000 | LR: 1.00000
Epoch  100 | Loss: 1.0214 | Acc: 0.4900 | LR: 0.90909
Epoch  200 | Loss: 0.9712 | Acc: 0.5333 | LR: 0.83333
Epoch  300 | Loss: 0.8545 | Acc: 0.5667 | LR: 0.76923
Epoch  400 | Loss: 0.7201 | Acc: 0.6400 | LR: 0.71429
Epoch  500 | Loss: 0.5894 | Acc: 0.7167 | LR: 0.66667
Epoch  600 | Loss: 0.5268 | Acc: 0.7600 | LR: 0.62500
Epoch  700 | Loss: 0.4746 | Acc: 0.7733 | LR: 0.58824
Epoch  800 | Loss: 0.4437 | Acc: 0.7967 | LR: 0.55556
Epoch  900 | Loss: 0.4223 | Acc: 0.8000 | LR: 0.52632
Epoch  999 | Loss: 0.4089 | Acc: 0.7967 | LR: 0.50025


In [9]:
#TEST 3 - AdaGrad
print("\n" + "="*70)
print(" Adaptive Gradient (AdaGrad)")
print("="*70)

# Reinitialize network
dense1 = Layer_Dense(2, 64)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 3)
activation2 = Activation_Softmax()
loss_function = Loss_CategoricalCrossEntropy()
optimizer = Optimizer_AdaGrad(learning_rate=1.0, decay=1e-3)

# Training loop
for epoch in range(1000):
    optimizer.pre_update_params()
    
    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    
    # Calculate loss and accuracy
    loss = loss_function.calculate(activation2.output, y)
    predictions = np.argmax(activation2.output, axis=1)
    accuracy = np.mean(predictions == y)
    
    if epoch % 100 == 0 or epoch == 999:
        print(f"Epoch {epoch:4d} | Loss: {loss:.4f} | Acc: {accuracy:.4f} | LR: {optimizer.current_learning_rate:.5f}")
    
    # Backward pass
    loss_function.backward(activation2.output, y)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()


 Adaptive Gradient (AdaGrad)
Epoch    0 | Loss: 1.0986 | Acc: 0.3333 | LR: 1.00000
Epoch  100 | Loss: 0.9538 | Acc: 0.5367 | LR: 0.90909
Epoch  200 | Loss: 0.8884 | Acc: 0.5900 | LR: 0.83333
Epoch  300 | Loss: 0.8495 | Acc: 0.6033 | LR: 0.76923
Epoch  400 | Loss: 0.8143 | Acc: 0.6300 | LR: 0.71429
Epoch  500 | Loss: 0.7825 | Acc: 0.6500 | LR: 0.66667
Epoch  600 | Loss: 0.7607 | Acc: 0.6467 | LR: 0.62500
Epoch  700 | Loss: 0.7379 | Acc: 0.6567 | LR: 0.58824
Epoch  800 | Loss: 0.7178 | Acc: 0.6767 | LR: 0.55556
Epoch  900 | Loss: 0.7023 | Acc: 0.6833 | LR: 0.52632
Epoch  999 | Loss: 0.6870 | Acc: 0.6467 | LR: 0.50025
